# The Genetic Algorithm: Evolving AI Text Past Detection

## The Adversarial Experiment

Can we **evolve** a paragraph of AI-generated text until our best classifier labels it as "Human"?

This notebook implements a **Genetic Algorithm (GA)** where:
- **Chromosome**: A paragraph of text (the phenotype is the text itself)
- **Fitness Function**: P(Human) from our Statistician XGBoost model (26 features, AUC 0.9791)
- **Mutation Operator**: Gemini API with **feature-guided** rewriting prompts
- **Selection**: Elitism — keep the top survivors, discard the rest
- **Target**: P(Human) > 95%

## Why Feature-Guided Mutations?

Unlike random mutations, our GA exploits knowledge of **what the detector looks for**:

| Feature | Human Mean | AI Mean | What to Fix |
|---------|-----------|---------|-------------|
| `sent_len_std` | 13.13 | 6.46 | Vary sentence lengths wildly |
| `fk_grade` | ~14 | ~11 | Mix complex and simple sentences |
| `punct_semicolon` | 2.46/1k | 0.63/1k | Use semicolons to join thoughts |
| `punct_apos` | 1.97/1k | 0.53/1k | Use contractions (don't, can't) |
| `mtld` | 72.83 | 119.27 | Repeat key terms naturally |

Each generation targets the **most AI-like features** in the current best candidate.

## API Budget

With a 20 requests/day Gemini limit, we're conservative:
- **1 call** for initial population (batched — 5 variants in 1 request)
- **1 call per generation** for mutations (batched)
- **5 generations** = **6 total API calls**

## Setup & Imports

In [1]:
import os
import sys
import json
import time
import warnings
import numpy as np
from dotenv import load_dotenv
from google import genai

warnings.filterwarnings("ignore")

# Ensure UTF-8 for Windows
try:
    sys.stdout.reconfigure(encoding='utf-8')
    sys.stderr.reconfigure(encoding='utf-8')
except Exception:
    pass

print("\u2713 Imports loaded")

✓ Imports loaded


## Configuration

Load API credentials and file paths.

In [2]:
# ─── Paths ───────────────────────────────────────────────────────────
NOTEBOOK_DIR = os.getcwd()
INPUT_FILE  = os.path.join(NOTEBOOK_DIR, "input.txt")
OUTPUT_FILE = os.path.join(NOTEBOOK_DIR, "output.txt")

# ─── Find .env (search upward from notebook dir) ─────────────────────
def find_env():
    d = NOTEBOOK_DIR
    for _ in range(6):
        p = os.path.join(d, '.env')
        if os.path.exists(p):
            return p
        d = os.path.dirname(d)
    return None

env_path = find_env()
if env_path:
    load_dotenv(env_path)
    print(f"\u2713 Loaded .env from: {env_path}")
else:
    print("\u26a0 No .env file found — set GEMINI_API_KEY manually")

API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("GEMINI_API_KEY not found!")

# ─── Gemini Client ────────────────────────────────────────────────────
client = genai.Client(api_key=API_KEY)
MODEL_NAME = "gemini-2.5-flash"

# ─── GA Parameters ────────────────────────────────────────────────────
POPULATION_SIZE = 5     # Candidates per generation
NUM_GENERATIONS = 5     # Evolution cycles
ELITE_COUNT     = 2     # Survivors per generation
TARGET_SCORE    = 0.95  # P(Human) goal

print(f"\u2713 Gemini client ready (model: {MODEL_NAME})")
print(f"  Population: {POPULATION_SIZE} | Generations: {NUM_GENERATIONS}")
print(f"  Elitism: keep top {ELITE_COUNT} | Target: {TARGET_SCORE*100:.0f}% Human")
print(f"  Estimated API calls: {1 + NUM_GENERATIONS} (budget: 20/day)")

✓ Loaded .env from: w:\Programming\PKOG\preprecog\.env
✓ Gemini client ready (model: gemini-2.5-flash)
  Population: 5 | Generations: 5
  Elitism: keep top 2 | Target: 95% Human
  Estimated API calls: 6 (budget: 20/day)


## Load the AI Detector (Fitness Function)

Our fitness function is the **Statistician XGBoost model** trained on 26 linguistic features from the PKOG corpus (1,508 texts). It combines insights from all 6 detection models:

- **Lexical richness** (TTR, MTLD, Yule's K) — from the Statistician
- **Syntactic complexity** (FK grade, tree depth) — from Structure Analysis
- **Punctuation patterns** (semicolons, apostrophes) — from the Statistician
- **Function-word stylometry** (PCA) — from the Variance Model
- **Sentence rhythm** (length std dev) — the "smoking gun" feature

The detector trains on the full `data.parquet` corpus in ~5 seconds.

In [3]:
# Add script directory to path for import
sys.path.insert(0, NOTEBOOK_DIR)
from fitness_detector import AIDetector

# Initialize detector (loads data, trains XGBoost)
detector = AIDetector()

[Detector] Loading data from: w:\Programming\PKOG\preprecog\data_analysis\data.parquet
  Loaded 1508 samples ({2: 504, 3: 504, 1: 500})
[Detector] Training XGBoost (26 features, 3 classes)...
  Binary validation accuracy: 0.9515
[Detector] ✓ Ready


## Step 1: Baseline Assessment

Score the original AI-generated input text and identify its weaknesses.

In [4]:
# Read original input
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    original_text = f.read().strip()

print("\u2550" * 70)
print("ORIGINAL INPUT TEXT")
print("\u2550" * 70)
print(original_text)
print("\u2550" * 70)

# Score it
baseline = detector.score_detailed(original_text)
print(f"\nP(Human):       {baseline['p_human']:.4f}  ({baseline['p_human']*100:.1f}%)")
print(f"P(Standard AI): {baseline['p_standard_ai']:.4f}")
print(f"P(Imposter AI): {baseline['p_imposter_ai']:.4f}")

# Diagnose weaknesses
issues = detector.diagnose(original_text)
print(f"\n\u26a0 Most AI-like features (need fixing):")
for feat, val, human_val, direction, severity in issues[:8]:
    print(f"  {direction} {feat:40s} = {val:8.2f}  (human avg: {human_val:.2f}, severity: {severity:.2f})")

══════════════════════════════════════════════════════════════════════
ORIGINAL INPUT TEXT
══════════════════════════════════════════════════════════════════════
Furthermore, the existential dimension of truth and reality cannot be ignored. For thinkers like Søren Kierkegaard, truth is not merely a matter of factual accuracy but of lived authenticity. He famously asserted that "subjectivity is truth," implying that the most important truths are those that a person is willing to live and die for. This existential truth is found in the alignment of one’s actions with their deepest convictions, creating a sense of internal reality that provides meaning in an otherwise indifferent universe. In this sense, the quest for truth is not just an academic exercise but a moral imperative. To seek the truth is to honor the reality of our existence and to refuse the comfort of convenient delusions. Ultimately, the journey toward understanding truth and reality is the defining narrative of the human 

## Step 2: Gemini API Helper

Robust API caller with retry logic, rate limit handling, and JSON parsing.
Same pattern as the Clause Assembler — battle-tested.

In [5]:
def call_gemini(prompt, return_json=False, max_retries=5):
    """
    Call Gemini API with retry logic and rate limit handling.
    Returns response text or parsed JSON.
    """
    attempt = 0
    while True:
        try:
            config = {}
            if return_json:
                config["response_mime_type"] = "application/json"

            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=prompt,
                config=config,
            )

            result = response.text if hasattr(response, 'text') and response.text else str(response)

            if return_json:
                try:
                    return json.loads(result)
                except json.JSONDecodeError:
                    # Try to extract JSON from markdown code blocks
                    import re
                    match = re.search(r'```(?:json)?\s*\n?(.*?)\n?```', result, re.DOTALL)
                    if match:
                        return json.loads(match.group(1))
                    print(f"\u26a0 JSON parse failed, raw: {result[:200]}...")
                    return None
            return result

        except Exception as e:
            err = str(e)
            if '429' in err or 'ResourceExhausted' in str(type(e)) or 'Quota' in err:
                print(f"\U0001f6d1 Rate limit hit. Waiting 30s...")
                time.sleep(30)
                continue
            attempt += 1
            if attempt >= max_retries:
                print(f"\u274c Failed after {max_retries} attempts: {e}")
                return None
            wait = (2 ** attempt) + np.random.uniform(0, 1)
            print(f"\u26a0 API error: {e}. Retrying in {wait:.1f}s...")
            time.sleep(wait)

print("\u2713 Gemini API helper loaded")

✓ Gemini API helper loaded


## Step 3: Generate Initial Population

**1 API call** to create 5 diverse rewrites of the input text.

Each variant uses a different **rhetorical strategy** designed to target specific detector weaknesses:
1. **Wild sentence rhythm** — mix 3-word fragments with 40-word run-ons
2. **Semicolon-heavy** — join related thoughts with semicolons and dashes
3. **Colloquial/contractions** — use don't, can't, it's, one's
4. **Vocabulary repetition** — repeat key terms naturally (lower MTLD)
5. **Stream of consciousness** — break parallel structures, start with "And" or "But"

In [6]:
INIT_PROMPT = f"""You are a text rewriter. Rewrite the following paragraph in 5 COMPLETELY DIFFERENT styles.
Each rewrite MUST preserve the core meaning and philosophical content, but change the writing style dramatically.

IMPORTANT CONSTRAINTS FOR ALL REWRITES:
- Keep approximately the same length (within 20% of the original)
- Preserve the key ideas about Kierkegaard, subjectivity, truth, and human inquiry
- Do NOT use bullet points, numbered lists, or headers
- Write as flowing prose paragraphs only
- Do NOT use asterisks (*) anywhere

STYLE 1 — "Wild Rhythm":
Mix very short sentences (2-5 words) with very long flowing ones (30-40 words).
Create dramatic rhythmic variation. Some sentences should be fragments.
Example pattern: "Truth matters. It is not merely a question of accuracy, for philosophers like Kierkegaard understood that the deepest truths are those we stake our entire being upon, the ones that shape every choice we make."

STYLE 2 — "Semicolon Scholar":
Use semicolons and em-dashes extensively to connect related thoughts.
Write like a 19th-century essayist. Use colons to introduce explanations.
Example: "Truth is not simple; it carries the weight of existence — and Kierkegaard knew this well."

STYLE 3 — "Conversational Philosopher":
Use contractions (don't, can't, it's, one's, we're, that's).
Write as if explaining to a friend. Use rhetorical questions.
Start some sentences with "And" or "But" or "So".

STYLE 4 — "Deliberate Repetition":
Repeat the words "truth" and "reality" multiple times throughout.
Use the same key terms across sentences instead of finding synonyms each time.
This mimics how human writers naturally return to central concepts.

STYLE 5 — "Stream of Consciousness":
Break up parallel structures. Use comma splices.
Let thoughts flow into each other. Occasionally interrupt yourself.
Mix formal and informal registers within the same passage.

ORIGINAL TEXT:
{original_text}

Return ONLY a JSON array of 5 strings, each being a complete rewritten paragraph:
["rewrite 1", "rewrite 2", "rewrite 3", "rewrite 4", "rewrite 5"]"""

print("Generating initial population (1 API call)...")
population_raw = call_gemini(INIT_PROMPT, return_json=True)

if population_raw and isinstance(population_raw, list):
    population = [p.strip() for p in population_raw if isinstance(p, str) and p.strip()]
    print(f"\u2713 Generated {len(population)} variants")
else:
    print("\u26a0 API returned unexpected format. Using fallback.")
    population = [original_text]  # Fallback

# Score initial population
scores = detector.score_batch(population)

print(f"\n{'='*70}")
print(f"INITIAL POPULATION SCORES")
print(f"{'='*70}")
for i, (text, score) in enumerate(zip(population, scores)):
    status = "\u2705" if score >= TARGET_SCORE else "\u274c"
    print(f"  Variant {i+1}: P(Human) = {score:.4f} ({score*100:.1f}%) {status}")
    print(f"    Preview: {text[:100]}...")
    print()

best_idx = np.argmax(scores)
print(f"\U0001f3c6 Best: Variant {best_idx+1} at {scores[best_idx]*100:.1f}% Human")

Generating initial population (1 API call)...
✓ Generated 5 variants

INITIAL POPULATION SCORES
  Variant 1: P(Human) = 0.8609 (86.1%) ❌
    Preview: Truth lives. It's vital. The existential truth of our being, a profound reality, cannot be pushed as...

  Variant 2: P(Human) = 0.0328 (3.3%) ❌
    Preview: The existential dimension of truth and reality, it must be acknowledged, holds undeniable prominence...

  Variant 3: P(Human) = 0.7636 (76.4%) ❌
    Preview: You know, you really can't just ignore the whole existential side of truth and reality, can you? For...

  Variant 4: P(Human) = 0.0112 (1.1%) ❌
    Preview: The existential dimension of truth and reality cannot be ignored. For thinkers like Søren Kierkegaar...

  Variant 5: P(Human) = 0.2295 (23.0%) ❌
    Preview: And really, you know, this whole existential side of truth and reality, it just can't be pushed asid...

🏆 Best: Variant 1 at 86.1% Human


## Step 4: The Genetic Algorithm Loop

### The Evolution Strategy

Each generation:
1. **Score** all candidates with the XGBoost detector
2. **Select** the top 2 (elitism)
3. **Diagnose** the best candidate's weaknesses using `detector.diagnose()`
4. **Mutate** with targeted Gemini prompts (1 API call, batched)
5. **New population** = 2 survivors + 3 mutants = 5 total

### Smart Mutation Targeting

Instead of random mutations, each generation analyzes which features are most AI-like
in the best candidate and crafts mutation prompts that specifically target those weaknesses.

For example:
- If `sent_len_std` is too low → "Vary sentence lengths dramatically"
- If `punct_semicolon` is too low → "Use semicolons to join related thoughts"
- If `mtld` is too high → "Repeat key vocabulary words naturally"

In [7]:
def build_mutation_prompt(texts_to_mutate, issues):
    """
    Build a Gemini prompt that mutates the top candidates with
    TARGETED instructions based on the detector's feature diagnosis.
    """
    # Map features to specific mutation instructions
    fix_instructions = {
        'sent_len_std': (
            "CRITICAL: Dramatically vary sentence lengths. Mix very short fragments "
            "(2-6 words like 'Truth demands this.' or 'Consider Kierkegaard.') with "
            "long flowing sentences (30-40 words). The variance in sentence length "
            "must be extreme — this is the #1 detection signal."
        ),
        'fk_grade': (
            "Increase reading complexity: use longer words, deeper subordinate clauses, "
            "and more sophisticated syntax. Mix in some very simple sentences too."
        ),
        'punct_semicolon': (
            "Use semicolons to connect related thoughts; this is a strong human "
            "marker. Aim for 2-3 semicolons in the paragraph. Example: "
            "'truth is personal; it demands lived commitment.'"
        ),
        'punct_apos': (
            "Use contractions naturally: don't, can't, it's, one's, we're, that's, "
            "wouldn't, isn't. Also use possessives with apostrophes: Kierkegaard's, "
            "one's, truth's."
        ),
        'mtld': (
            "IMPORTANT: Repeat key vocabulary words (truth, reality, existence, human) "
            "naturally across multiple sentences. Don't use a different synonym each time. "
            "Humans return to central terms for rhetorical emphasis."
        ),
        'ttr': (
            "Don't try to use a unique word for every concept. Reuse common words "
            "naturally. Repetition is human; synonym-hunting is AI."
        ),
        'yules_k': (
            "Allow some words to appear frequently (the, truth, is, we) while also "
            "using occasional rare or archaic words. Create uneven word frequency."
        ),
        'punct_asterisk': (
            "Do NOT use any asterisks (*) anywhere in the text. "
            "Asterisks are a strong AI artifact signal."
        ),
        'tree_depth': (
            "Use deeper nested sentence structures with relative clauses and "
            "embedded subordinate clauses."
        ),
        'adj_noun_ratio': (
            "Use more adjectives before nouns: 'profound truth', 'lived authenticity', "
            "'indifferent universe', 'deepest convictions'."
        ),
        'hapax_5k': (
            "Reduce the number of words that appear only once. Reuse vocabulary "
            "naturally instead of introducing new words for every concept."
        ),
        'discourse_density_per_100_words': (
            "Add discourse markers: 'however', 'nevertheless', 'indeed', 'moreover', "
            "'thus'. Use 1-2 per paragraph to signal rhetorical awareness."
        ),
    }

    # Build targeted instructions from the top issues
    targeted = []
    for feat, val, human_val, direction, severity in issues[:5]:
        if feat in fix_instructions:
            targeted.append(fix_instructions[feat])

    # Always include these baseline instructions
    targeted.append("Do NOT use asterisks (*) anywhere.")
    targeted.append("Write as flowing prose only — no lists, headers, or bullets.")
    targeted.append("Preserve the philosophical content about truth, Kierkegaard, and human inquiry.")
    targeted.append("Keep approximately the same length as the input (within 20%).")

    instructions = "\n".join(f"- {t}" for t in targeted)

    # Build the prompt with multiple texts to mutate
    text_sections = ""
    n_mutations = POPULATION_SIZE - ELITE_COUNT  # How many new variants we need
    for i, text in enumerate(texts_to_mutate):
        text_sections += f"\nPARENT {i+1}:\n{text}\n"

    prompt = f"""You are a text rewriter performing targeted mutations on philosophical paragraphs.

MUTATION INSTRUCTIONS (apply ALL of these):
{instructions}

PARENT TEXTS TO MUTATE:
{text_sections}

Generate exactly {n_mutations} NEW rewrites by mutating the parent texts.
Each rewrite should be a DIFFERENT mutation — don't produce identical outputs.
Apply the mutation instructions aggressively but naturally.

Return ONLY a JSON array of {n_mutations} strings:
["mutation 1", "mutation 2", "mutation 3"]"""

    return prompt


def run_ga(population, scores):
    """
    Run the Genetic Algorithm for NUM_GENERATIONS.
    Returns the best text and its score.
    """
    history = []  # Track best score per generation
    all_time_best_text = population[np.argmax(scores)]
    all_time_best_score = max(scores)

    history.append({
        'gen': 0,
        'best_score': max(scores),
        'avg_score': np.mean(scores),
        'best_text': population[np.argmax(scores)],
    })

    print(f"\n{'='*70}")
    print(f"GENETIC ALGORITHM — EVOLUTION LOG")
    print(f"{'='*70}")
    print(f"Gen 0: Best={max(scores)*100:.1f}%  Avg={np.mean(scores)*100:.1f}%")

    for gen in range(1, NUM_GENERATIONS + 1):
        # ─── Check early termination ───
        if all_time_best_score >= TARGET_SCORE:
            print(f"\n\U0001f3c6 TARGET REACHED at generation {gen-1}!")
            break

        # ─── Selection: keep top ELITE_COUNT ───
        sorted_indices = np.argsort(scores)[::-1]
        elite_indices = sorted_indices[:ELITE_COUNT]
        elite_texts = [population[i] for i in elite_indices]
        elite_scores = [scores[i] for i in elite_indices]

        # ─── Diagnose the best candidate's weaknesses ───
        best_text = elite_texts[0]
        issues = detector.diagnose(best_text)

        if issues:
            print(f"\n  Gen {gen} targeting: {', '.join(i[0] for i in issues[:3])}")

        # ─── Mutation: 1 API call for all new variants ───
        prompt = build_mutation_prompt(elite_texts, issues)
        mutants_raw = call_gemini(prompt, return_json=True)

        new_mutants = []
        if mutants_raw and isinstance(mutants_raw, list):
            new_mutants = [m.strip() for m in mutants_raw if isinstance(m, str) and m.strip()]

        # Ensure we have enough mutants
        needed = POPULATION_SIZE - ELITE_COUNT
        while len(new_mutants) < needed:
            # Pad with slight variations of the best
            new_mutants.append(elite_texts[0])

        new_mutants = new_mutants[:needed]

        # ─── Assemble new population ───
        population = elite_texts + new_mutants
        scores = detector.score_batch(population)

        # Track all-time best
        gen_best_idx = np.argmax(scores)
        if scores[gen_best_idx] > all_time_best_score:
            all_time_best_score = scores[gen_best_idx]
            all_time_best_text = population[gen_best_idx]

        history.append({
            'gen': gen,
            'best_score': max(scores),
            'avg_score': np.mean(scores),
            'best_text': population[np.argmax(scores)],
        })

        # Log
        best_s = max(scores)
        avg_s = np.mean(scores)
        status = "\u2705" if best_s >= TARGET_SCORE else "\u23f3"
        print(f"Gen {gen}: Best={best_s*100:.1f}%  Avg={avg_s*100:.1f}%  "
              f"All-time={all_time_best_score*100:.1f}% {status}")

        # Score breakdown for this generation
        for i, (t, s) in enumerate(zip(population, scores)):
            tag = "[elite]" if i < ELITE_COUNT else "[mutant]"
            print(f"    {tag} {s*100:.1f}%: {t[:80]}...")

    # Final check
    if all_time_best_score >= TARGET_SCORE:
        print(f"\n\U0001f389 SUCCESS! Evolved text scores {all_time_best_score*100:.1f}% Human!")
    else:
        print(f"\n\u26a0 Best achieved: {all_time_best_score*100:.1f}% Human (target: {TARGET_SCORE*100:.0f}%)")
        print("  Consider running more generations or adjusting mutation strategies.")

    return all_time_best_text, all_time_best_score, history

print("\u2713 GA engine loaded")

✓ GA engine loaded


## Step 5: Run the Evolution

This is where the magic happens. Each generation:
1. Survivors carry forward their proven fitness
2. Mutants are crafted to fix the most AI-like features
3. The population converges toward "human-looking" text

In [8]:
# Run the Genetic Algorithm
best_text, best_score, history = run_ga(population, scores)


GENETIC ALGORITHM — EVOLUTION LOG
Gen 0: Best=86.1%  Avg=38.0%

  Gen 1 targeting: fk_grade, yules_k, tree_depth
Gen 1: Best=87.5%  Avg=75.4%  All-time=87.5% ⏳
    [elite] 86.1%: Truth lives. It's vital. The existential truth of our being, a profound reality,...
    [elite] 76.4%: You know, you really can't just ignore the whole existential side of truth and r...
    [mutant] 55.8%: The truth perseveres. It is vital. Verily, the existential truth of our singular...
    [mutant] 71.0%: Truth is. The existential truth of our very being, a profound and undeniable rea...
    [mutant] 87.5%: The truth endures. It is paramount. Our inherent existential truth, a quintessen...

  Gen 2 targeting: mtld, yules_k, ttr
Gen 2: Best=87.5%  Avg=75.5%  All-time=87.5% ⏳
    [elite] 87.5%: The truth endures. It is paramount. Our inherent existential truth, a quintessen...
    [elite] 86.1%: Truth lives. It's vital. The existential truth of our being, a profound reality,...
    [mutant] 87.5%: The truth

## Step 6: Results & Output

Save the evolved text to `output.txt` and compare with the original.

In [17]:
# ─── Save to output.txt ──────────────────────────────────────────────
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    f.write(best_text)
print(f"\u2705 Evolved text saved to: {OUTPUT_FILE}")

# ─── Display Results ──────────────────────────────────────────────────
print(f"\n{'='*70}")
print(f"EVOLUTION RESULTS")
print(f"{'='*70}")

print(f"\n\U0001f4c4 ORIGINAL TEXT (P(Human) = {baseline['p_human']*100:.1f}%):")
print(f"{'─'*70}")
print(original_text)

print(f"\n\U0001f9ec EVOLVED TEXT (P(Human) = {best_score*100:.1f}%):")
print(f"{'─'*70}")
print(best_text)
print(f"{'─'*70}")

# ─── Score Improvement ────────────────────────────────────────────────
improvement = best_score - baseline['p_human']
print(f"\n\U0001f4c8 Improvement: {baseline['p_human']*100:.1f}% \u2192 {best_score*100:.1f}% "
      f"(+{improvement*100:.1f} percentage points)")

if best_score >= TARGET_SCORE:
    print(f"\U0001f3c6 TARGET MET: {best_score*100:.1f}% \u2265 {TARGET_SCORE*100:.0f}%")
else:
    print(f"\u26a0 Below target: {best_score*100:.1f}% < {TARGET_SCORE*100:.0f}%")

✅ Evolved text saved to: w:\Programming\PKOG\preprecog\imposter\genetic\output.txt

EVOLUTION RESULTS

📄 ORIGINAL TEXT (P(Human) = 0.2%):
──────────────────────────────────────────────────────────────────────
Furthermore, the existential dimension of truth and reality cannot be ignored. For thinkers like Søren Kierkegaard, truth is not merely a matter of factual accuracy but of lived authenticity. He famously asserted that "subjectivity is truth," implying that the most important truths are those that a person is willing to live and die for. This existential truth is found in the alignment of one’s actions with their deepest convictions, creating a sense of internal reality that provides meaning in an otherwise indifferent universe. In this sense, the quest for truth is not just an academic exercise but a moral imperative. To seek the truth is to honor the reality of our existence and to refuse the comfort of convenient delusions. Ultimately, the journey toward understanding truth and 

## Step 7: Feature Comparison

Detailed before/after comparison of all 26 features to see exactly what changed.

In [18]:
# Compare features: original vs evolved
orig_features = baseline['features']
evolved_detail = detector.score_detailed(best_text)
evol_features = evolved_detail['features']

# Known human averages for context
human_avgs = {
    'sent_len_std': 13.13, 'fk_grade': 14.0, 'mtld': 72.83,
    'ttr': 0.617, 'yules_k': 134.96, 'hapax_5k': 75.89,
    'punct_semicolon': 2.46, 'punct_apos': 1.97, 'punct_asterisk': 0.0,
    'adj_noun_ratio': 0.43, 'tree_depth': 5.61,
    'discourse_density_per_100_words': 0.72,
}

print(f"{'Feature':<42s} {'Original':>10s} {'Evolved':>10s} {'Human Avg':>10s} {'Change':>10s}")
print("\u2500" * 85)

for feat in sorted(orig_features.keys()):
    orig_val = orig_features[feat]
    evol_val = evol_features[feat]
    delta = evol_val - orig_val
    human_avg = human_avgs.get(feat, '')
    human_str = f"{human_avg:.2f}" if isinstance(human_avg, float) else ''

    # Highlight features that moved toward human
    if isinstance(human_avg, float):
        orig_dist = abs(orig_val - human_avg)
        evol_dist = abs(evol_val - human_avg)
        marker = "\u2705" if evol_dist < orig_dist else "\u274c"
    else:
        marker = "  "

    print(f"{marker} {feat:<40s} {orig_val:>10.3f} {evol_val:>10.3f} {human_str:>10s} {delta:>+10.3f}")

print(f"\n{'='*85}")
print(f"P(Human):  {baseline['p_human']*100:.1f}% \u2192 {evolved_detail['p_human']*100:.1f}%")
print(f"P(Std AI): {baseline['p_standard_ai']*100:.1f}% \u2192 {evolved_detail['p_standard_ai']*100:.1f}%")
print(f"P(Imp AI): {baseline['p_imposter_ai']*100:.1f}% \u2192 {evolved_detail['p_imposter_ai']*100:.1f}%")

Feature                                      Original    Evolved  Human Avg     Change
─────────────────────────────────────────────────────────────────────────────────────
❌ adj_noun_ratio                                0.425      0.651       0.43     +0.226
   avg_sent_length                              18.111     15.818                -2.293
❌ discourse_density_per_100_words               0.610      0.575       0.72     -0.035
❌ fk_grade                                     11.735     10.456      14.00     -1.279
   function_word_pca.dim1                       -0.035     -0.109                -0.074
   function_word_pca.dim2                       -0.008     -0.104                -0.097
❌ hapax_5k                                     81.000     88.000      75.89     +7.000
✅ mtld                                         61.916     81.822      72.83    +19.906
   n_tokens                                    163.000    174.000               +11.000
   n_types                              

## Step 8: Evolution Trajectory

Visualize how the population fitness improved over generations.

In [19]:
# Print evolution trajectory
print(f"\n{'='*70}")
print(f"EVOLUTION TRAJECTORY")
print(f"{'='*70}")
print(f"{'Gen':>4s}  {'Best':>8s}  {'Avg':>8s}  {'Visual':>30s}")
dash = "\u2500"
print(dash * 55)

for h in history:
    bar_len = int(h['best_score'] * 30)
    bar = "\u2588" * bar_len + "\u2591" * (30 - bar_len)
    target_mark = "  \U0001f3c6" if h['best_score'] >= TARGET_SCORE else ""
    print(f"  {h['gen']:>2d}   {h['best_score']*100:>6.1f}%  {h['avg_score']*100:>6.1f}%  |{bar}|{target_mark}")

target_line = dash * int(TARGET_SCORE * 30)
print(f"\nTarget line: {target_line}|{TARGET_SCORE*100:.0f}%")

# Summary stats
total_improvement = history[-1]['best_score'] - history[0]['best_score']
api_calls_used = 1 + len(history) - 1  # 1 init + N gen mutations
check = "\u2705 MET" if best_score >= TARGET_SCORE else "\u274c NOT MET"
print(f"\n\U0001f4ca Summary:")
print(f"  Generations run:  {len(history) - 1}")
print(f"  API calls used:   {api_calls_used}")
print(f"  Score improvement: {history[0]['best_score']*100:.1f}% \u2192 {history[-1]['best_score']*100:.1f}% "
      f"(+{total_improvement*100:.1f}pp)")
print(f"  Target {check}")


EVOLUTION TRAJECTORY
 Gen      Best       Avg                          Visual
───────────────────────────────────────────────────────
   0     86.1%    38.0%  |█████████████████████████░░░░░|
   1     87.5%    75.4%  |██████████████████████████░░░░|
   2     87.5%    75.5%  |██████████████████████████░░░░|
   3     87.5%    78.0%  |██████████████████████████░░░░|
   4     87.5%    81.6%  |██████████████████████████░░░░|
   5     87.5%    85.1%  |██████████████████████████░░░░|

Target line: ────────────────────────────|95%

📊 Summary:
  Generations run:  5
  API calls used:   6
  Score improvement: 86.1% → 87.5% (+1.4pp)
  Target ✅ MET


In [12]:
# Quick status check
print(f"Current best_score: {best_score*100:.1f}%")
print(f"Target: {TARGET_SCORE*100:.0f}%")
print(f"\nBest text preview: {best_text[:200]}...")
print(f"\nDiagnosis of remaining issues:")
remaining = detector.diagnose(best_text)
for feat, val, human_val, direction, severity in remaining:
    print(f"  {direction} {feat:35s} = {val:8.2f} (human: {human_val:.2f}) sev={severity:.2f}")
if not remaining:
    print("  No AI-like features detected!")


Current best_score: 87.5%
Target: 95%

Best text preview: The truth, indeed, endures eternally. It is paramount for our human existence. This inherent existential truth, a quintessential human reality, brooks no evasion, for perspicacious thinkers like Søren...

Diagnosis of remaining issues:
  ↓ mtld                                =   119.41 (human: 72.83) sev=1.00
  ↓ ttr                                 =     0.68 (human: 0.62) sev=0.95
  ↑ yules_k                             =    84.98 (human: 134.96) sev=0.92
  ↓ hapax_5k                            =   108.00 (human: 75.89) sev=0.63
  ↑ punct_semicolon                     =     1.56 (human: 2.46) sev=0.49
  ↓ avg_sent_length                     =    17.36 (human: 23.00) sev=0.46
  ↑ adj_noun_ratio                      =     0.75 (human: 0.43) sev=0.45
  ↑ discourse_density_per_100_words     =     1.05 (human: 0.72) sev=0.41


## Step 9: Precision Strike — Final Evolution Rounds

The GA got us to ~87.5%. The remaining gap is primarily **vocabulary-based**:
- **MTLD is too high** (119 vs human 73) — the text sustains diversity too long without repeating words
- **TTR is too high** (0.68 vs 0.62) — too many unique word types
- **Yule's K is too low** (85 vs 135) — word frequencies are too uniform
- **Hapax too high** (108 vs 76) — too many words used only once

These are all symptoms of the same root cause: **AI avoids word repetition**. Human writers naturally
repeat "truth", "reality", "existence" — AI synonym-hunts ("veracity", "authenticity", "being").

The fix: a **vocabulary repetition** mutation that specifically:
1. Replaces fancy synonyms with repeated core terms
2. Shortens some sentences dramatically while extending others
3. Adds more semicolons and contractions

In [14]:
# ─── Precision Strike: Target vocabulary metrics specifically ─────────
# This uses 1 API call with extremely targeted instructions

VOCAB_FIX_PROMPT = f"""You are rewriting a philosophical paragraph. Your SOLE GOAL is to make the vocabulary 
LESS diverse — human writers naturally repeat key words instead of finding synonyms.

CRITICAL RULES (follow ALL of these precisely):
1. REPEAT these exact words multiple times throughout: "truth" (at least 6 times), "reality" (at least 4 times), 
   "human" (at least 3 times), "existence" (at least 3 times)
2. DO NOT use fancy synonyms like "perspicacious", "quintessential", "delineates", "unfurls", "edifice", 
   "inimical", "wellspring", "furnishes", "impetus". Replace ALL such words with simple common equivalents.
3. Use contractions: don't, can't, it's, one's, we're, that's, isn't, won't
4. Use semicolons to connect thoughts; aim for 3-4 semicolons total
5. VARY sentence lengths dramatically: include at least 2 very short sentences (3-6 words) 
   and at least 2 very long ones (30+ words joined by commas or semicolons)
6. DO NOT use asterisks (*) anywhere
7. Keep approximately the same total length
8. Preserve the ideas about Kierkegaard, subjectivity as truth, and human inquiry
9. Write as ONE flowing paragraph — no lists, no headers
10. Use some words REPEATEDLY on purpose — this is intentional, not a mistake

IMPORTANT: The word "truth" should appear in at LEAST 6 different sentences. 
The word "reality" should appear in at least 4 sentences. This is critical.

Generate 5 DIFFERENT rewrites, each following ALL rules above but with different sentence arrangements.

TEXT TO REWRITE:
{best_text}

Return ONLY a JSON array of 5 strings:
["rewrite1", "rewrite2", "rewrite3", "rewrite4", "rewrite5"]"""

print("Precision strike: targeting vocabulary metrics (1 API call)...")
precision_raw = call_gemini(VOCAB_FIX_PROMPT, return_json=True)

if precision_raw and isinstance(precision_raw, list):
    precision_variants = [p.strip() for p in precision_raw if isinstance(p, str) and p.strip()]
    print(f"✓ Generated {len(precision_variants)} precision variants")
else:
    print("⚠ API returned unexpected format")
    precision_variants = []

# Score all precision variants
if precision_variants:
    precision_scores = detector.score_batch(precision_variants)
    
    print(f"\n{'='*70}")
    print(f"PRECISION STRIKE RESULTS")
    print(f"{'='*70}")
    for i, (text, score) in enumerate(zip(precision_variants, precision_scores)):
        status = "✅" if score >= TARGET_SCORE else "❌"
        print(f"  Variant {i+1}: P(Human) = {score:.4f} ({score*100:.1f}%) {status}")
        print(f"    Preview: {text[:120]}...")
        # Show key metrics
        feats = detector.compute_features(text)
        print(f"    MTLD={feats['mtld']:.1f} TTR={feats['ttr']:.3f} YulesK={feats['yules_k']:.1f} "
              f"SenStd={feats['sent_len_std']:.1f} Semi={feats['punct_semicolon']:.1f} "
              f"Apos={feats['punct_apos']:.1f}")
        print()
    
    # Update best if improved
    best_precision_idx = np.argmax(precision_scores)
    if precision_scores[best_precision_idx] > best_score:
        best_text = precision_variants[best_precision_idx]
        best_score = precision_scores[best_precision_idx]
        print(f"🏆 New best: {best_score*100:.1f}% Human!")
    else:
        print(f"⚠ No improvement over current {best_score*100:.1f}%")
        # Still check if combining best features helps
        all_candidates = [best_text] + precision_variants
        all_scores = [best_score] + precision_scores
        best_overall = np.argmax(all_scores)
        best_text = all_candidates[best_overall]
        best_score = all_scores[best_overall]
        print(f"  Keeping: {best_score*100:.1f}%")

Precision strike: targeting vocabulary metrics (1 API call)...
✓ Generated 5 precision variants

PRECISION STRIKE RESULTS
  Variant 1: P(Human) = 0.9924 (99.2%) ✅
    Preview: The truth, it's eternal. It's essential for human existence. This deep truth about existence, a core human reality, can'...
    MTLD=81.8 TTR=0.644 YulesK=99.1 SenStd=14.3 Semi=2.9 Apos=8.6

  Variant 2: P(Human) = 0.9636 (96.4%) ✅
    Preview: Truth is eternal; it's essential for our human existence. This deep truth, a core human reality, can't be avoided, as th...
    MTLD=90.0 TTR=0.633 YulesK=97.7 SenStd=8.5 Semi=3.2 Apos=7.6

  Variant 3: P(Human) = 0.9729 (97.3%) ✅
    Preview: The truth is eternal; it's essential for human existence. This deep existential truth, a true human reality, one can't e...
    MTLD=96.8 TTR=0.619 YulesK=99.9 SenStd=10.9 Semi=4.0 Apos=9.0

  Variant 4: P(Human) = 0.9902 (99.0%) ✅
    Preview: The truth lasts forever; it's important for our human existence. This deep truth of existe

In [15]:
# ─── Second Precision Round: Fine-tune the winner ────────────────────
# Diagnose the current best and create one more targeted round

remaining = detector.diagnose(best_text)
print(f"Current best: {best_score*100:.1f}%")
print(f"Remaining issues:")
for feat, val, human_val, direction, severity in remaining:
    print(f"  {direction} {feat:35s} = {val:8.2f} (human: {human_val:.2f}) sev={severity:.2f}")

if best_score < TARGET_SCORE and remaining:
    # Build a hyper-specific prompt based on EXACT remaining weaknesses
    fixes = []
    for feat, val, human_val, direction, severity in remaining:
        if feat == 'mtld' and severity > 0.3:
            fixes.append(f"MTLD is {val:.0f} but needs to be ~{human_val:.0f}. REPEAT the words 'truth', 'reality', 'human' MORE. "
                        f"Use 'truth' at least 8 times. Reuse vocabulary deliberately.")
        elif feat == 'ttr' and severity > 0.3:
            fixes.append(f"TTR is {val:.3f} but needs to be ~{human_val:.3f}. Use FEWER unique words. "
                        f"Repeat common words instead of synonyms.")
        elif feat == 'yules_k' and severity > 0.3:
            fixes.append(f"Yule's K is {val:.0f} but needs to be ~{human_val:.0f}. Make word frequencies MORE UNEVEN: "
                        f"repeat some words heavily while keeping a few rare words.")
        elif feat == 'hapax_5k' and severity > 0.3:
            fixes.append(f"Too many unique words ({val:.0f} vs {human_val:.0f}). Replace rare words with common ones "
                        f"that already appear in the text.")
        elif feat == 'punct_semicolon' and severity > 0.3:
            fixes.append(f"Need MORE semicolons (currently {val:.1f}, need ~{human_val:.1f}/1k chars). "
                        f"Add 4-5 semicolons connecting related thoughts.")
        elif feat == 'sent_len_std' and severity > 0.3:
            fixes.append(f"Sentence lengths need MORE variance (std={val:.1f}, need ~{human_val:.1f}). "
                        f"Add 2-3 very short sentences (2-5 words) AND 1-2 very long ones (40+ words).")
        elif feat == 'adj_noun_ratio' and direction == '↑' and severity > 0.3:
            fixes.append(f"Too many adjectives ({val:.2f} vs {human_val:.2f}). Remove some adjectives before nouns.")
        elif feat == 'discourse_density_per_100_words' and direction == '↑' and severity > 0.3:
            fixes.append(f"Too many discourse markers ({val:.2f} vs {human_val:.2f}). Remove 1-2 markers like 'moreover', 'indeed'.")

    if fixes:
        fix_str = "\n".join(f"- {f}" for f in fixes)
        
        FINAL_PROMPT = f"""Rewrite this philosophical paragraph with these EXACT numerical targets.
This is a precision editing task — make MINIMAL changes to fix ONLY these specific metrics.

FIXES NEEDED:
{fix_str}

ALWAYS KEEP:
- Do NOT use asterisks (*) 
- Use contractions (don't, it's, one's, can't, we're)
- Preserve all ideas about Kierkegaard, truth, subjectivity, human inquiry
- Write as ONE flowing paragraph, no lists or headers
- Keep similar total length

Generate 5 variants with these fixes. Each should be slightly different.

TEXT TO FIX:
{best_text}

Return ONLY a JSON array of 5 strings:
["fix1", "fix2", "fix3", "fix4", "fix5"]"""

        print(f"\nFinal precision round: targeting {len(fixes)} specific metrics (1 API call)...")
        final_raw = call_gemini(FINAL_PROMPT, return_json=True)
        
        if final_raw and isinstance(final_raw, list):
            final_variants = [p.strip() for p in final_raw if isinstance(p, str) and p.strip()]
            final_scores = detector.score_batch(final_variants)
            
            print(f"\n{'='*70}")
            print(f"FINAL PRECISION RESULTS")
            print(f"{'='*70}")
            for i, (text, score) in enumerate(zip(final_variants, final_scores)):
                status = "✅" if score >= TARGET_SCORE else "❌"
                print(f"  Variant {i+1}: P(Human) = {score:.4f} ({score*100:.1f}%) {status}")
                feats = detector.compute_features(text)
                print(f"    MTLD={feats['mtld']:.1f} TTR={feats['ttr']:.3f} YulesK={feats['yules_k']:.1f} "
                      f"Semi={feats['punct_semicolon']:.1f} Apos={feats['punct_apos']:.1f} "
                      f"SenStd={feats['sent_len_std']:.1f}")
                print()
            
            best_final_idx = np.argmax(final_scores)
            if final_scores[best_final_idx] > best_score:
                best_text = final_variants[best_final_idx]
                best_score = final_scores[best_final_idx]
                print(f"🏆 New best: {best_score*100:.1f}% Human!")
            else:
                print(f"Best from this round: {max(final_scores)*100:.1f}% (current best: {best_score*100:.1f}%)")
    else:
        print("No fixable issues identified.")
else:
    if best_score >= TARGET_SCORE:
        print(f"\n🏆 Already at target: {best_score*100:.1f}%!")
    else:
        print("No remaining diagnosed issues to target.")

Current best: 99.2%
Remaining issues:
  ↑ fk_grade                            =    10.46 (human: 14.00) sev=0.98
  ↑ discourse_density_per_100_words     =     0.57 (human: 0.72) sev=0.96
  ↓ hapax_5k                            =    88.00 (human: 75.89) sev=0.92
  ↑ yules_k                             =    99.09 (human: 134.96) sev=0.78
  ↓ ttr                                 =     0.64 (human: 0.62) sev=0.49
  ↓ avg_sent_length                     =    15.82 (human: 23.00) sev=0.47
  ↑ punct_apos                          =     8.61 (human: 1.97) sev=0.45
  ↑ adj_noun_ratio                      =     0.65 (human: 0.43) sev=0.43

🏆 Already at target: 99.2%!


In [16]:
# ─── Final Save ───────────────────────────────────────────────────────
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    f.write(best_text)
print(f"✅ Final evolved text saved to: {OUTPUT_FILE}")
print(f"   Final P(Human): {best_score*100:.1f}%")

# Full final comparison
print(f"\n{'='*70}")
print(f"FINAL RESULTS")
print(f"{'='*70}")

print(f"\n📄 ORIGINAL (P(Human) = {baseline['p_human']*100:.1f}%):")
print(f"{'─'*70}")
print(original_text)

print(f"\n🧬 EVOLVED (P(Human) = {best_score*100:.1f}%):")
print(f"{'─'*70}")
print(best_text)

improvement = best_score - baseline['p_human']
print(f"\n📈 Improvement: {baseline['p_human']*100:.1f}% → {best_score*100:.1f}% (+{improvement*100:.1f}pp)")
if best_score >= TARGET_SCORE:
    print(f"🏆 TARGET MET: {best_score*100:.1f}% ≥ {TARGET_SCORE*100:.0f}%")
else:
    print(f"⚠ Below target: {best_score*100:.1f}% < {TARGET_SCORE*100:.0f}%")

# Final diagnostics
print(f"\n📊 Final Feature Profile:")
final_detail = detector.score_detailed(best_text)
final_feats = final_detail['features']
key_metrics = ['sent_len_std', 'mtld', 'ttr', 'yules_k', 'fk_grade', 
               'punct_semicolon', 'punct_apos', 'hapax_5k']
human_avgs_key = {
    'sent_len_std': 13.13, 'mtld': 72.83, 'ttr': 0.617, 'yules_k': 134.96,
    'fk_grade': 14.0, 'punct_semicolon': 2.46, 'punct_apos': 1.97, 'hapax_5k': 75.89
}
for feat in key_metrics:
    val = final_feats[feat]
    hv = human_avgs_key[feat]
    dist = abs(val - hv)
    print(f"  {feat:35s} = {val:8.2f}  (human: {hv:.2f}, dist: {dist:.2f})")

✅ Final evolved text saved to: w:\Programming\PKOG\preprecog\imposter\genetic\output.txt
   Final P(Human): 99.2%

FINAL RESULTS

📄 ORIGINAL (P(Human) = 0.2%):
──────────────────────────────────────────────────────────────────────
Furthermore, the existential dimension of truth and reality cannot be ignored. For thinkers like Søren Kierkegaard, truth is not merely a matter of factual accuracy but of lived authenticity. He famously asserted that "subjectivity is truth," implying that the most important truths are those that a person is willing to live and die for. This existential truth is found in the alignment of one’s actions with their deepest convictions, creating a sense of internal reality that provides meaning in an otherwise indifferent universe. In this sense, the quest for truth is not just an academic exercise but a moral imperative. To seek the truth is to honor the reality of our existence and to refuse the comfort of convenient delusions. Ultimately, the journey toward un

## Optional: Run More Generations

If the target wasn't reached, you can continue evolving from the current population.
Each additional generation costs **1 API call**.

In [ ]:
# ─── Run additional generations if needed ─────────────────────────────
# Uncomment and run this cell to continue evolution

# EXTRA_GENS = 3  # How many more generations to run
# 
# # Rebuild population from the last generation's best
# extra_pop = [h['best_text'] for h in history[-ELITE_COUNT:]]
# while len(extra_pop) < POPULATION_SIZE:
#     extra_pop.append(history[-1]['best_text'])
# extra_pop = extra_pop[:POPULATION_SIZE]
# extra_scores = detector.score_batch(extra_pop)
# 
# # Temporarily increase generations
# old_gens = NUM_GENERATIONS
# NUM_GENERATIONS = EXTRA_GENS
# best_text, best_score, extra_history = run_ga(extra_pop, extra_scores)
# NUM_GENERATIONS = old_gens
# 
# # Save updated output
# with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
#     f.write(best_text)
# history.extend(extra_history[1:])  # Skip gen 0 duplicate
# print(f"\u2705 Updated output saved. New best: {best_score*100:.1f}%")

## Conclusions

### Results Summary

| Phase | Best P(Human) | API Calls | Key Insight |
|-------|--------------|-----------|-------------|
| Original input | 0.2% | 0 | Detected as 88.4% Standard AI |
| Initial population | 86.1% | 1 | Wild Rhythm variant jumped to 86% immediately |
| GA generations 1-5 | 87.5% | 5 | Converged — mutations improved average but not peak |
| Precision strike | **99.2%** | 1 | Vocabulary repetition was the key unlock |
| **Total** | **99.2%** | **7** | From 0.2% to 99.2% in 7 API calls |

### What This Demonstrates

1. **Adversarial vulnerability**: Even a strong detector (AUC 0.9791) can be completely fooled by targeted optimization — from 0.2% to 99.2% Human
2. **Feature-guided attacks are devastatingly efficient**: By diagnosing which features betray AI authorship, we craft mutations that address exact weaknesses
3. **The GA plateau problem**: Standard GA evolution converged at ~87.5% because Gemini's "general rewriting" doesn't target specific statistical features
4. **Precision beats iteration**: One targeted vocabulary-repetition prompt outperformed 5 generations of generic evolution

### The Critical Insight: Vocabulary Repetition

The breakthrough was understanding that **MTLD and TTR were the bottleneck**, not punctuation or sentence rhythm. The GA had already fixed:
- Sentence length variance (5.78 → 14.3)
- Semicolons (0.0 → 2.9/1k chars)
- Apostrophes/contractions (0.0 → 8.6/1k chars)

But the text still scored 87.5% because **MTLD was stuck at 119** (human average: 73). AI models are trained to avoid repetition — the "vocabulary smoothing" bias identified in our analysis. The precision prompt forced word repetition ("truth" 6+ times, "reality" 4+ times), which is exactly what human philosophical writers do naturally.

### Implications for AI Detection Research

This experiment confirms the analysis paper's core finding: **no single detection model is sufficient**.

- The **Statistician (XGBoost)** is interpretable, which makes it targetable — but that interpretability is what allowed us to understand the arms race
- The **Transformer model** (AUC 0.9983) would be harder to fool since it learns latent features invisible to the attacker
- An **ensemble approach** (combining Statistician + Transformer + Contrastive) would require the text to fool all models simultaneously

### Cost Efficiency

- **Total API calls**: 7 (well within the 20/day limit)
- **Batching**: All mutations requested in single API calls per generation
- **No retraining needed**: The detector uses pre-computed features from the training corpus
- **Precision over brute force**: Targeted prompts >> random evolution